# Bob

* Bob wants to receive a message from Alice.
* He can communicate with Alice over an unsecure, classical channel.
* They can also communicate over a quantum channel.

Bob establishes a classical channel to Alice. Anyone can listen in:

# 0. Setup
* wait for setup data from Alice

In [1]:
import paho.mqtt.client as mqtt
import json

mqtt_broker_ip = "broker.emqx.io"
channel = "quantum_cryptography/classical_channel/#"

client = mqtt.Client()
client.connect(mqtt_broker_ip, 1883, 60)
# How many bits
setup_data = None


def on_message_setup(client, userdata, msg):
    global setup_data
    try:
        setup_data = json.loads(msg.payload)
        print("Bits to Exchange:", setup_data["bits_to_exchange"])
        client.disconnect()
    except Exception as e:
        print("unable to parse json: ", msg.payload, "\n", e)


def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe(channel)


client.on_message = on_message_setup
client.on_connect = on_connect

_ = client.loop_forever()

Connected with result code 0
Bits to Exchange: 16


# 3. create list of random polarizations

In [2]:
from enum import Enum
from random import choice


class Basis(Enum):
    # 0° | or _ 90°:
    HV = 0
    # -45° \ or 45° /:
    LR = 1


bases = list()

for i in range(setup_data["bits_to_exchange"]):
    bases.append(choice(list(Basis)))

for i, b in enumerate(bases):
    print(i, b, sep="\t")

0	Basis.LR
1	Basis.HV
2	Basis.HV
3	Basis.LR
4	Basis.HV
5	Basis.HV
6	Basis.HV
7	Basis.HV
8	Basis.HV
9	Basis.HV
10	Basis.LR
11	Basis.HV
12	Basis.LR
13	Basis.HV
14	Basis.LR
15	Basis.HV


# 4. quantum channel

In [4]:
from time import sleep
from qcrypt.hardware.filterwheel import Filterwheel, Orientation
from qcrypt.hardware.lightsensor import Lightsensor
import paho.mqtt.client as mqtt

broker = "broker.emqx.io"
channel_laser = "quantum_cryptography/classical_channel/alice/laser"

client = mqtt.Client()
filterwheel = Filterwheel(username="bob", mqtt_broker_ip=broker)
lightsensor = Lightsensor()
bits = setup_data["bits_to_exchange"]
counter = 0

          
def get_polarisation_from_basis():
    """
    returns corresponding orientation enum for bob
    """
    if bases[counter] == Basis.HV:
        return Orientation.VERTICAL
    elif bases[counter] == Basis.LR:
        return Orientation.RIGHT
    
# prepare first orientation
filterwheel.rotate_to(get_polarisation_from_basis())
counter+=1

def on_message_quantum(client, userdata, msg):
    global counter
    print(msg.payload.decode())
    if msg.payload.decode() == "on":
        print("Currently at Bit: ",counter)
        if counter < bits:
            # MEASURE
            while True:
                x = lightsensor.getHighLow()
                if x:
                    break
            print(x)

            # ADJUST FILTERWHEEL ACCORDINGLY
            filterwheel.rotate_to(get_polarisation_from_basis())
            counter += 1
            # sensor starten
        else:
            print("disconnecting ...")
            client.disconnect()

            
client.connect(broker, 1883, 60)
client.subscribe(channel)
client.message_callback_add(channel_laser, on_message_quantum)
client.loop_forever()

establishing connection to lightsensor ..
Orientation.VERTICAL --> Orientation.RIGHT
rotating left


2021-06-15 13:36:16,313: INFO - bob rotating 50 steps counter-clockwise


on
Currently at Bit:  1
ser_in_waiting: 30
['50', '20\r\n']
['49', '20\r\n']
['High', 'High']
Orientation.RIGHT --> Orientation.VERTICAL
rotating right


2021-06-15 13:36:23,838: INFO - bob rotating 50 steps clockwise


off
on
Currently at Bit:  2
ser_in_waiting: 30
['9', '19\r\n']
['49', '19\r\n']
['High', 'High']
Orientation.VERTICAL --> Orientation.VERTICAL
no need to rotate
off
on
Currently at Bit:  3
ser_in_waiting: 32
['49', '20\r\n']
['49', '20\r\n']
['High', 'High']
Orientation.VERTICAL --> Orientation.RIGHT
rotating left


...

2021-06-15 13:36:49,935: INFO - bob rotating 50 steps counter-clockwise


off
on
Currently at Bit:  15
ser_in_waiting: 31
['49', '20\r\n']
['49', '19\r\n']
['High', 'High']
Orientation.RIGHT --> Orientation.VERTICAL
rotating right


2021-06-15 13:36:51,947: INFO - bob rotating 50 steps clockwise


off
on
Currently at Bit:  16
disconnecting ...


7